In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wholesale-customer-data/Wholesale customers data.csv


# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

# Loading Dataset

In [4]:
df=pd.read_csv("/kaggle/input/wholesale-customer-data/Wholesale customers data.csv")

In [5]:
df.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


In [6]:
df.shape

(440, 8)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Channel           440 non-null    int64
 1   Region            440 non-null    int64
 2   Fresh             440 non-null    int64
 3   Milk              440 non-null    int64
 4   Grocery           440 non-null    int64
 5   Frozen            440 non-null    int64
 6   Detergents_Paper  440 non-null    int64
 7   Delicassen        440 non-null    int64
dtypes: int64(8)
memory usage: 27.6 KB


In [9]:
df.isnull().sum()

Channel             0
Region              0
Fresh               0
Milk                0
Grocery             0
Frozen              0
Detergents_Paper    0
Delicassen          0
dtype: int64

In [10]:
x = df.drop("Channel", axis=1)
y = df["Channel"]

In [28]:
df["Channel"].unique()

array([0, 1])

In [11]:
x

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,3,12669,9656,7561,214,2674,1338
1,3,7057,9810,9568,1762,3293,1776
2,3,6353,8808,7684,2405,3516,7844
3,3,13265,1196,4221,6404,507,1788
4,3,22615,5410,7198,3915,1777,5185
...,...,...,...,...,...,...,...
435,3,29703,12051,16027,13135,182,2204
436,3,39228,1431,764,4510,93,2346
437,3,14531,15488,30243,437,14841,1867
438,3,10290,1981,2232,1038,168,2125


In [12]:
y

0      2
1      2
2      2
3      1
4      2
      ..
435    1
436    1
437    2
438    1
439    1
Name: Channel, Length: 440, dtype: int64

In [13]:
y.unique()

array([2, 1])

In [14]:
# converting labels into binary
y[y == 2] = 0
y[y == 1] = 1

In [15]:
y

0      0
1      0
2      0
3      1
4      0
      ..
435    1
436    1
437    0
438    1
439    1
Name: Channel, Length: 440, dtype: int64

Now we will convert dataset into optimized data structure called **DMatrix**.

# What is DMatrix?
In the context of XGBoost, the DMatrix is a special optimized data structure used to store the dataset for training and prediction. It’s designed to be faster and more memory-efficient than a standard pandas DataFrame or NumPy array.
## Why Use DMatrix?
- Optimized for performance: It’s faster in training/predicting.
- Supports missing values handling.
- Allows specifying weights, labels, and other metadata.
- Works well with large datasets (can even load from disk).

In [16]:
import xgboost as xgb
# define DMatrix
data_matrix = xgb.DMatrix(data=x, label=y)

# Spliting Dataset

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Model Training 

In [20]:
# importing XGBClassifier
from xgboost import XGBClassifier

**Parameters for XGB Classifer**

1. **Basic Parameters** 
| Parameter      | Description                                                                                                                                                                                  |
| -------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `objective`    | Learning task. Common ones: <br> - `'binary:logistic'` (binary classification) <br> - `'multi:softprob'` (multi-class, returns probs) <br> - `'multi:softmax'` (multi-class, returns labels) |
| `booster`      | Booster type to use: <br> - `'gbtree'` (tree-based)  default <br> - `'gblinear'` (linear model) <br> - `'dart'` (dropout trees)                                                             |
| `n_estimators` | Number of boosting rounds (trees)                                                                                                                                                            |
| `verbosity`    | Print messages (0: silent, 1: warning, 2: info, 3: debug)                                                                                                                                    |

2. **Tree Parameters**
| Parameter          | Description                                                                       |
| ------------------ | --------------------------------------------------------------------------------- |
| `max_depth`        | Max depth of each tree (controls complexity)                                      |
| `min_child_weight` | Minimum sum of instance weight (hessian) needed in a child → prevents overfitting |
| `gamma`            | Minimum loss reduction required to make a split → higher = more conservative      |
| `subsample`        | Fraction of training data used for each tree → helps avoid overfitting (e.g. 0.8) |
| `colsample_bytree` | Fraction of features used per tree                                                |

3. **Boosting Parameters**
| Parameter                  | Description                                                                           |
| -------------------------- | ------------------------------------------------------------------------------------- |
| `learning_rate` (or `eta`) | Shrinks weight of each tree (typical: 0.01–0.3)                                       |
| `scale_pos_weight`         | Used for imbalanced classes (e.g., 1 for balanced, >1 for more positive class weight) |
| `early_stopping_rounds`    | Used with validation set — stops training if no improvement after N rounds            |

4. **Regularization Parameters**
| Parameter    | Description                                    |
| ------------ | ---------------------------------------------- |
| `reg_alpha`  | L1 regularization term on weights (like Lasso) |
| `reg_lambda` | L2 regularization term on weights (like Ridge) |


In [22]:
# declaring parameters
params = {
    'objective':'binary:logistic',
    'max_depth': 4,
    'reg_alpha': 10,
    'learning_rate' : 1.0,
    'n_estimators': 100
}

# instantiate the classifier
classifier = XGBClassifier(**params)
classifier.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1.0, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [23]:
# view the parameters of xgb trained model as follows
print(classifier)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1.0, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)


In [25]:
# prediction on testing dataset
y_pred = classifier.predict(x_test)
y_pred

array([1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1])

In [27]:
from sklearn.metrics import accuracy_score, f1_score

acs = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average = 'binary')

print("Accuracy Score: ", acs)
print("F1 Score: ", f1)

Accuracy Score:  0.8977272727272727
F1 Score:  0.928


# What is Cross-Validation?
Cross-validation is a technique in machine learning used to:

- Evaluate the performance of a model.
- Ensure the model generalizes well to unseen data (i.e., avoids overfitting).
- Get a more reliable estimate of accuracy, F1 score, etc.

## Why Not Just train_test_split?
Using train_test_split() gives you performance based on just one split. That can be:
- Biased if the split isn’t representative.
- Unreliable for small datasets.

Cross-validation solves this by using multiple train-test splits.

## How Cross-Validation Works (K-Fold CV)
K-Fold Cross-Validation is the most common type:

1. Split your dataset into K equal parts (folds).
2. For each fold:
- Train on K-1 folds.
- Test on the remaining fold.

3. Repeat this process K times, each time with a different fold as the test set.

4. Average the results (accuracy, F1, etc.).

```
Fold 1: Train on [2,3,4,5], Test on [1] 
Fold 2: Train on [1,3,4,5], Test on [2]
...
Fold 5: Train on [1,2,3,4], Test on [5]
```

## CV Parameters
| Parameter                   | Description                                                                  |
| --------------------------- | ---------------------------------------------------------------------------- |
| **`dtrain`**                | `DMatrix` version of training data (faster and memory-efficient for XGBoost) |
| **`params`**                | Dictionary of hyperparameters (objective, regularization, etc.)              |
| **`objective`**             | `'binary:logistic'`: for binary classification (outputs probabilities)       |
| **`colsample_bytree`**      | Randomly selects a fraction of features per tree (helps regularization)      |
| **`learning_rate`**         | Shrinks contribution of each tree. Smaller is slower but more stable         |
| **`max_depth`**             | Limits how complex each tree can get                                         |
| **`alpha`**                 | L1 regularization: drives some weights to zero (like Lasso)                  |
| **`nfold`**                 | Number of folds to split the training data (e.g., 3-fold CV)                 |
| **`num_boost_round`**       | Max number of boosting rounds (early stopping may halt earlier)              |
| **`early_stopping_rounds`** | Stop training if score doesn't improve for `n` rounds                        |
| **`metrics`**               | Evaluation metric during CV — `auc`, `error`, `logloss`, etc.                |
| **`as_pandas`**             | If `True`, return results as a `pandas.DataFrame`                            |
| **`seed`**                  | Fixes the randomness for reproducibility                                     |


In [32]:
from xgboost import cv

params = {"objective":"binary:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

xgb_cv = cv(dtrain=data_matrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10, metrics="auc", as_pandas=True, seed=123)

In [34]:
xgb_cv.head()

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.916130,0.017352,0.883424,0.031286
1,0.916677,0.017711,0.884176,0.031818
2,0.942047,0.010679,0.921339,0.004898
3,0.959857,0.004367,0.947506,0.010233
4,0.963008,0.002962,0.944536,0.010897


In [41]:
# get overall mean AUC score from test dataset
overall_test_auc = xgb_cv['test-auc-mean'].max()

# print the overall score
print(f"Overall Test AUC score: {overall_test_auc}")

Overall Test AUC score: 0.9560670309317983
